In [1]:
# input = """x00: 1
# x01: 1
# x02: 1
# y00: 0
# y01: 1
# y02: 0

# x00 AND y00 -> z00
# x01 XOR y01 -> z01
# x02 OR y02 -> z02"""


# input = """x00: 1
# x01: 0
# x02: 1
# x03: 1
# x04: 0
# y00: 1
# y01: 1
# y02: 1
# y03: 1
# y04: 1

# ntg XOR fgs -> mjb
# y02 OR x01 -> tnw
# kwq OR kpj -> z05
# x00 OR x03 -> fst
# tgd XOR rvg -> z01
# vdt OR tnw -> bfw
# bfw AND frj -> z10
# ffh OR nrd -> bqk
# y00 AND y03 -> djm
# y03 OR y00 -> psh
# bqk OR frj -> z08
# tnw OR fst -> frj
# gnj AND tgd -> z11
# bfw XOR mjb -> z00
# x03 OR x00 -> vdt
# gnj AND wpb -> z02
# x04 AND y00 -> kjc
# djm OR pbm -> qhw
# nrd AND vdt -> hwm
# kjc AND fst -> rvg
# y04 OR y02 -> fgs
# y01 AND x02 -> pbm
# ntg OR kjc -> kwq
# psh XOR fgs -> tgd
# qhw XOR tgd -> z09
# pbm OR djm -> kpj
# x03 XOR y03 -> ffh
# x00 XOR y04 -> ntg
# bfw OR bqk -> z06
# nrd XOR fgs -> wpb
# frj XOR qhw -> z04
# bqk OR frj -> z07
# y03 OR x01 -> nrd
# hwm AND bqk -> z03
# tgd XOR rvg -> z12
# tnw OR pbm -> gnj"""

input = open("inputs/24").read()

In [2]:
def parse_circuit(input_str):
    # Initialize result dictionaries
    values = {}
    operations = {}

    # Process each line
    for line in input_str.splitlines():
        if not line:
            continue

        if "->" in line:
            # Handle operation lines
            inputs, output = line.split(" -> ")
            parts = inputs.split()
            operations[output] = (parts[1], parts[0], parts[2])
        else:
            # Handle value assignments
            name, value = line.split(": ")
            values[name] = bool(int(value))

    return values, operations


def swap_strings(text, str1, str2):
    """
    Simultaneously swaps two strings in a text.

    Args:
        text (str): Input text containing strings to be swapped
        str1 (str): First string to swap
        str2 (str): Second string to swap

    Returns:
        str: Text with strings swapped
    """
    # Use a temporary placeholder that's unlikely to appear in the text
    TEMP_PLACEHOLDER = f"__TEMP_PLACEHOLDER_{hash(str1)}_{hash(str2)}__"

    # First replacement: str1 -> TEMP
    text = text.replace(str1, TEMP_PLACEHOLDER)

    # Second replacement: str2 -> str1
    text = text.replace(str2, str1)

    # Final replacement: TEMP -> str2
    text = text.replace(TEMP_PLACEHOLDER, str2)

    return text


In [3]:
start_values, gates = parse_circuit(input)

In [4]:
gates

{'bss': ('AND', 'y16', 'x16'),
 'fjs': ('OR', 'wrt', 'pss'),
 'tdb': ('AND', 'qpd', 'hnk'),
 'ktn': ('AND', 'ncp', 'drd'),
 'kgt': ('AND', 'vnw', 'ftq'),
 'dbg': ('XOR', 'y12', 'x12'),
 'jfq': ('AND', 'y05', 'x05'),
 'grr': ('XOR', 'x16', 'y16'),
 'tbr': ('AND', 'y39', 'x39'),
 'qkk': ('AND', 'crp', 'gkk'),
 'jrf': ('AND', 'x02', 'y02'),
 'nwn': ('AND', 'x09', 'y09'),
 'rjw': ('AND', 'vmf', 'bkp'),
 'z34': ('XOR', 'qpd', 'hnk'),
 'kcc': ('XOR', 'x06', 'y06'),
 'bgj': ('AND', 'x30', 'y30'),
 'rmv': ('AND', 'rhk', 'btv'),
 'z44': ('XOR', 'fjs', 'bmv'),
 'z25': ('XOR', 'btv', 'rhk'),
 'vnc': ('OR', 'wgk', 'ppp'),
 'fkb': ('XOR', 'kcm', 'grr'),
 'fbb': ('AND', 'fkb', 'rcc'),
 'z15': ('XOR', 'dbd', 'shb'),
 'vsq': ('XOR', 'y38', 'x38'),
 'qpd': ('OR', 'sfs', 'hmh'),
 'kwh': ('AND', 'bmv', 'fjs'),
 'z11': ('XOR', 'ftq', 'vnw'),
 'kcm': ('OR', 'cjt', 'svk'),
 'ttn': ('XOR', 'y33', 'x33'),
 'dwd': ('XOR', 'x18', 'y18'),
 'nkn': ('OR', 'qpk', 'btq'),
 'mfc': ('OR', 'nnq', 'pfb'),
 'z39': ('XOR'

In [5]:
from itertools import chain


In [6]:
from collections import defaultdict


def make_edgelist(gates):
    edgelist = defaultdict(list)
    operations = {}  # Store operations for each output->input pair

    # for output, (op, a, b) in gates.items():
    #     edgelist[a].append(output)
    #     edgelist[b].append(output)

    for output, (op, a, b) in gates.items():
        edgelist[a].append(output)
        edgelist[b].append(output)
        # Store the operation type for each input->output connection
        operations[(a, output)] = op
        operations[(b, output)] = op

    return edgelist, operations

In [7]:
import networkx as nx

edgelist, operations = make_edgelist(gates)

# Create DiGraph and add edges with operation attributes
G = nx.DiGraph()
for source, targets in edgelist.items():
    for target in targets:
        G.add_edge(source, target, op=operations[(source, target)])

n_nodes = G.number_of_nodes()
topo_order = list(nx.topological_sort(G))
for idx, node in enumerate(topo_order):
    G.nodes[node]["topo_order"] = idx / (n_nodes - 1)


def label_type(s):
    if s[0] == "x":
        return "x"
    elif s[0] == "y":
        return "y"
    elif s[0] == "z":
        return "z"
    else:
        return "gate"


for node in G.nodes:
    G.nodes[node]["type"] = label_type(node)

# Calculate path distances FROM z00 by reversing the graph
G_reversed = G.reverse()
path_lengths = nx.shortest_path_length(G_reversed, source="z00")

# Add distances as node attributes in original graph
for node in G.nodes:
    # If there's no path from z00 to this node, set distance to -1
    G.nodes[node]["dist_from_z00"] = path_lengths.get(node, -1)


In [8]:
G.number_of_nodes(), G.number_of_edges()

(312, 444)

In [9]:
nx.write_graphml(G, "graph.graphml")

In [10]:
def topo_sort(graph):
    # NOTE: no cycle detection!
    order = []
    all_nodes = set(graph.keys()) | set(chain(*graph.values()))
    unvisited = all_nodes

    def dfs(node):
        if node in unvisited:
            unvisited.remove(node)

        for neighbor in graph[node]:
            if neighbor in unvisited:
                dfs(neighbor)

        # we've explored all of node's dependencies: safe to append
        order.append(node)

    while unvisited:
        start_node = unvisited.pop()
        dfs(start_node)

    return order

In [11]:
topo_order = topo_sort(edgelist)[::-1]
topo_order

['x13',
 'y24',
 'x03',
 'y25',
 'x17',
 'x11',
 'y11',
 'qmm',
 'x23',
 'y38',
 'y26',
 'x40',
 'y04',
 'y09',
 'x24',
 'y05',
 'x22',
 'y31',
 'x20',
 'x01',
 'x42',
 'y41',
 'x02',
 'y20',
 'x41',
 'x43',
 'x39',
 'y27',
 'y02',
 'y34',
 'x19',
 'x15',
 'y01',
 'x32',
 'y07',
 'y22',
 'btq',
 'y44',
 'x31',
 'tjk',
 'hbc',
 'y23',
 'nbr',
 'x09',
 'jtc',
 'y08',
 'y10',
 'y37',
 'x04',
 'y42',
 'kcp',
 'x10',
 'vmf',
 'vsm',
 'x05',
 'jfq',
 'x07',
 'y30',
 'x18',
 'x28',
 'y00',
 'y43',
 'pss',
 'dnf',
 'y29',
 'y28',
 'hdk',
 'y36',
 'x16',
 'cbh',
 'jnj',
 'x37',
 'z37',
 'rqw',
 'y40',
 'wqg',
 'ntw',
 'y21',
 'wsm',
 'x21',
 'nnr',
 'rqf',
 'y35',
 'y14',
 'ncp',
 'x33',
 'y18',
 'x36',
 'ckw',
 'fjn',
 'y12',
 'gtv',
 'fpv',
 'crt',
 'mtb',
 'x14',
 'y39',
 'gkk',
 'tbr',
 'kmf',
 'y03',
 'pps',
 'mgr',
 'dwg',
 'gcg',
 'jrf',
 'x26',
 'mcj',
 'qfv',
 'y13',
 'dgr',
 'cpr',
 'y15',
 'shb',
 'svk',
 'y06',
 'pct',
 'x29',
 'knv',
 'ccs',
 'sqr',
 'fvc',
 'x00',
 'prt',
 'cdb',


In [12]:
def num_to_padded_binary(num, pad_length=45):
    start = bin(num)[2:]

    if len(start) > pad_length:
        raise ValueError("Number too large to fit in padded binary")

    return start.zfill(pad_length)


def binary_string_to_bool_list(binary_string):
    return [bool(int(c)) for c in binary_string]


binary_string_to_bool_list(num_to_padded_binary(123))[:5]

[False, False, False, False, False]

In [13]:
def binary_string_from_z_values(value_dict):
    # Filter keys starting with 'z' and sort in descending order
    z_items = [(k, v) for k, v in value_dict.items() if k.startswith("z")]
    z_items.sort(reverse=True)

    # Convert booleans to string of 1s and 0s
    binary_string = "".join(str(int(v)) for _, v in z_items)

    return binary_string

    # return binary_string_to_bool_list(binary_string)

    # Convert binary string to integer
    # return int(binary_string, 2)


# int(binary_string_from_z_values(value_dict), 2)

In [14]:
def simulate_circuit(value_dict, gates):
    edgelist, _ = make_edgelist(gates)
    topo_order = topo_sort(edgelist)[::-1]

    for node in topo_order:
        if node in value_dict:
            continue

        if node in gates:
            op, a, b = gates[node]

            if op == "AND":
                value_dict[node] = value_dict[a] and value_dict[b]
            elif op == "OR":
                value_dict[node] = value_dict[a] or value_dict[b]
            elif op == "XOR":
                value_dict[node] = value_dict[a] != value_dict[b]

    return binary_string_from_z_values(value_dict)


int(simulate_circuit(start_values.copy(), gates), 2)

53325321422566

In [15]:
num_to_padded_binary(1)

'000000000000000000000000000000000000000000001'

In [28]:
def make_start_dict(x_input, y_input):
    # least significant bit first so we reverse
    x_bin = num_to_padded_binary(x_input)[::-1]
    y_bin = num_to_padded_binary(y_input)[::-1]
    # x_bin = num_to_padded_binary(x_input)
    # y_bin = num_to_padded_binary(y_input)

    return {f"x{i:02}": bool(int(x_bin[i])) for i in range(len(x_bin))} | {
        f"y{i:02}": bool(int(y_bin[i])) for i in range(len(y_bin))
    }


# fkb z16?

# input_a = 1
# input_b = 2

# input_a = 234903
# input_b = 940209
input_a = 2**44 + 1
input_b = 2**44 - 1

# input_a = 2**45 - 11
# input_b = 2**45 - 11

correct_answer = num_to_padded_binary(input_a + input_b, pad_length=46)
naive_answer = simulate_circuit(make_start_dict(input_a, input_b), gates)

Methodology: try additions and look from least to most significant bit for errors
Visualize the graph in Gephi and stare to find the right swaps

In [40]:
# fkb and z16
# rqf and nnr
# rdn and z31

modified_input = swap_strings(input, "-> fkb", "-> z16")
modified_input = swap_strings(modified_input, "-> rqf", "-> nnr")
modified_input = swap_strings(modified_input, "-> rdn", "-> z31")
modified_input = swap_strings(modified_input, "-> rrn", "-> z37")
_, modified_gates = parse_circuit(modified_input)

In [41]:
modified_answer = simulate_circuit(make_start_dict(input_a, input_b), modified_gates)

In [42]:
print(naive_answer)
print(correct_answer)
print(modified_answer)

1000000001111110000000000111110000000000000000
1000000000000000000000000000000000000000000000
1000000000000000000000000000000000000000000000


In [50]:
swaps = ["fkb", "z16", "rqf", "nnr", "rdn", "z31", "rrn", "z37"]
print(",".join(sorted(swaps)))

fkb,nnr,rdn,rqf,rrn,z16,z31,z37
